### Importing needed libaries

In [14]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import os
import numpy as np
import torch.nn as nn
from torch.nn import functional as F

### Download data

In [15]:
!wget --no-check-certificate 'https://www.dropbox.com/scl/fi/kkrfmiskhap3rxvh6vv8b/pb3_train.txt?rlkey=13xaop0af6c7xhhwtpzfimuqc&dl=0' -O ./data_train.txt

with open('data_train.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2024-03-01 14:40:37--  https://www.dropbox.com/scl/fi/kkrfmiskhap3rxvh6vv8b/pb3_train.txt?rlkey=13xaop0af6c7xhhwtpzfimuqc&dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uca9c5a8ac5affb71066bf375384.dl.dropboxusercontent.com/cd/0/inline/COSo4c2W3w5XfVSebkWLQIFxo5IfuN7jL-BYdIDCyfpA4COIrxtKcH525FqbYzICk40YOoUDw1-sSt5ZvgCJCjyDPZXmiKB8Ral0h6hLiW7i5wPnikXypwBw16Gm3Xgr7FQuzYITKzqRl2ZQfuC1fzsw/file# [following]
--2024-03-01 14:40:38--  https://uca9c5a8ac5affb71066bf375384.dl.dropboxusercontent.com/cd/0/inline/COSo4c2W3w5XfVSebkWLQIFxo5IfuN7jL-BYdIDCyfpA4COIrxtKcH525FqbYzICk40YOoUDw1-sSt5ZvgCJCjyDPZXmiKB8Ral0h6hLiW7i5wPnikXypwBw16Gm3Xgr7FQuzYITKzqRl2ZQfuC1fzsw/file
Resolving uca9c5a8ac5affb71066bf375384.dl.dropboxusercontent.com (uca9c5a8ac5affb71066bf375384.dl.dropboxusercontent.com)... 162.125.80.

### Tokenizer

In [16]:
class Tokenizer:
    ''' Tokenizer class '''

    def __init__(self, vocab):
        ''' Initialize tokenizer '''

        self.vocab = vocab
        self.vocab_size = len(vocab)
        self.stoi = { ch:i for i,ch in enumerate(vocab) }
        self.itos = { i:ch for i,ch in enumerate(vocab) }
        self.encode = lambda s: [self.stoi[c] for c in s]
        self.decode = lambda l: ''.join([self.itos[i] for i in l])

# Load vocabulary
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    raw = f.read()
vocab = sorted(set(raw))


# Create tokenizer
tokenizer = Tokenizer(vocab)
print('Vocab size:', tokenizer.vocab_size)

--2024-03-01 14:40:40--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  6.08MB/s    in 0.2s    

2024-03-01 14:40:40 (6.08 MB/s) - ‘input.txt.1’ saved [1115394/1115394]

Vocab size: 65


### Hyper-parameters

In [17]:
# TODO: choose hyper-parameters
batch_size = 32 # How many samples to process at each iteration (DO NOT CHANGE)
block_size = 128  # Maximum context length (DO NOT CHANGE)
max_iters = 20000 # Maximum number of iterations - change later -
eval_interval = 100 # How often to evaluate the model
learning_rate =  0.003 # Learning rate - change later -  1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Device (accelerator)
eval_iters = 200 # Number of steps to evaluate the model
n_embd = 64 # Embedding dimension (should be divisible by n_head) - change later
n_head = 4 # Number of heads - change later
n_layer = 4 # Number of layers - change later
dropout = 0.01 # Dropout probability [0, 1] - change later

In [18]:
print(device)

cuda


### Train and test splits

In [19]:
''' Data tokenization and splits '''
# TODO: complete the code to encode the text by the tokenizer
tokenized_text = tokenizer.encode(text)
data = torch.tensor(tokenized_text, dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

### Get batch of data for training

In [20]:
def get_batch(split):
    ''' Generate a small batch of data of inputs x and targets y '''

    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    # TODO: complete the code to implement next token prediction by defining y
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

### Estimate loss

In [21]:
@torch.no_grad()
def estimate_loss():
    ''' Estimate loss '''

    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

### Transformer Modules

In [22]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        # TODO: Complete the code by defining the key, query and value as Linear operations without bias
        # Hint: dimensions of the linear operations must be n_embd x head_size
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        # TODO: Transform the input x by apply the key and query operations to get k and q
        k = self.key(x)
        q = self.query(x)

        # TODO: Implement the scaled self-attention mechanism by multiplying keys and queries
        # Hint: scale by multiplying by C ** -0.5 and use .transpose(-2, -1).
        A = q @ k.transpose(-2,-1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)

        A = A.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        # TODO: Apply softmax function to A by setting dim=-1
        # Hint: use the function F.softmax
        A = F.softmax(A, dim=-1)


        A = self.dropout(A)
        # TODO: transform the input x to get the values
        v = self.value(x) # b, t, c
        # TODO: Multiply values by A to obtain the output
        # Hint: the output should have the same shape as input x i.e. (B, T, C)
        out = A @ v   # (b, t, t).(b, t, c)

        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        ''' n_embd: embedding dimension, n_head: the number of heads '''

        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        # TODO: Implement the forward pass by applying the following operations:
        # 1) apply layer norm ln1 to x
        # 2) apply self-attention sa to the result in step 1)
        # 3) add skip connection by adding the input x to the result in step 2)
        x = x + self.sa(self.ln1(x))
        # TODO: Implement the forward pass by applying the following operations:
        # a) apply layer norm ln2 to the result in step 3)
        # b) apply feed-forward ffwd to the result in step a)
        # c) add skip connection by adding the input x to the result in step b)
        x = x + self.ffwd(self.ln2(x))
        return x

### Transformer Architecture

In [23]:
class Transformer(nn.Module):
    ''' A Transformer model '''

    def __init__(self, n_layer):
        ''' n_layer: the number of Transformer blocks '''

        super().__init__()
        self.token_embedding_table = nn.Embedding(tokenizer.vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, tokenizer.vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # TODO: define tok_emb by applying token_embedding_table to the input idx
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        # TODO: add tok_emb and pos_emb to get x
        x = tok_emb + pos_emb
        # TODO: apply the blocks to get the output
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            # TODO: compute the cross entropy loss
            loss = F.cross_entropy(logits, targets)

        return logits, loss

### Define model

In [24]:
model = Transformer(n_layer=n_layer)

### Training loop

In [ ]:
m = model.to(device)
print(sum(p.numel() for p in m.parameters()), 'M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
val_loss = []
for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        val_loss.append(losses['val'])

    # TODO: get a batch of training samples by calling the function get_batch
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    # TODO: backpropagate on the loss
    loss.backward()
    optimizer.step()

215873 M parameters
step 0: train loss 4.3157, val loss 4.3139
step 100: train loss 2.6260, val loss 2.6627
step 200: train loss 2.4993, val loss 2.5401
step 300: train loss 2.4413, val loss 2.4867
step 400: train loss 2.3846, val loss 2.4292
step 500: train loss 2.3178, val loss 2.3771
step 600: train loss 2.2508, val loss 2.3139
step 700: train loss 2.2043, val loss 2.2676
step 800: train loss 2.1481, val loss 2.2175
step 900: train loss 2.0910, val loss 2.1677
step 1000: train loss 2.0379, val loss 2.1237
step 1100: train loss 1.9888, val loss 2.0834
step 1200: train loss 1.9481, val loss 2.0525
step 1300: train loss 1.9086, val loss 2.0200
step 1400: train loss 1.8765, val loss 1.9930
step 1500: train loss 1.8390, val loss 1.9704
step 1600: train loss 1.8199, val loss 1.9561
step 1700: train loss 1.7907, val loss 1.9419
step 1800: train loss 1.7665, val loss 1.9239
step 1900: train loss 1.7498, val loss 1.9170
step 2000: train loss 1.7260, val loss 1.9050
step 2100: train loss 1.71

In [ ]:
# plot validation loss
plt.plot(val_loss)
plt.xlabel('step')
plt.ylabel('val loss')
plt.show()

In [ ]:
# Save final model checkpoint and architecture
torch.save(model, 'model.pth')